In [ ]:
# s = 0, t = 8
def BFS(s, t, parent, flowMatrix):
    # boolean array to keep track of visited nodes
    # [False, False, , ..] - 9 elements
    visited = [False] * (t + 1)
    queue = []

    queue.append(s)
    # [True, False, , .. ,False]
    visited[s] = True

    # continues as long as there are nodes in the queue to explore.
    while queue:
        # pops the first node from the queue to explore its neighbors.
        u = queue.pop(0)
        # flowMatrix[u] is [0, 2, 2, 2, 3, 0, 0, 0, 0]
        # enumerate(flowMatrix[u]) iterates through the elements of flowMatrix[u]
        for index, value in enumerate(flowMatrix[u]):
            if visited[index] == False and value > 0:
                queue.append(index)
                visited[index] = True
                parent[index] = u

                '''
                If the target node t is reached (if index == t:), the function returns True,
                indicating that there is an augmenting path from the source to the target. Otherwise,
                it continues searching for augmenting paths.
                '''
                if index == t:
                    return True

    # If no augmenting path is found in the BFS traversal, the function returns False,
    # indicating that there are no more augmenting paths from s to t.
    return False

# sink = sizeofflowmatrix-1 = 8
def FordFulkerson(flowMatrix, sink):
    source = 0

    parent = [-1] * (sink + 1)

    max_flow = 0

    while BFS(source, sink, parent, flowMatrix):
        # initializes a variable to store the minimum flow in the augmenting path
        # It's set to positive infinity to ensure that any flow value on the path will be smaller.
        path_flow = float("Inf")
        s = sink

        # loop to find the minimum cut of the augmenting path
        while s != source:
            path_flow = min(path_flow, flowMatrix[parent[s]][s])
            s = parent[s]

        max_flow += path_flow
        v = sink

        while v != source:
            u = parent[v]
            # along the augmenting path minimize the minimum cut or pathflow from the edge capacity - forward flow
            flowMatrix[u][v] -= path_flow
            # reverse flow so add
            flowMatrix[v][u] += path_flow
            v = parent[v]

    return max_flow

peopleCount = int(input())
committeesCount = int(input())

peoples = list(map(str, input().split()))
committees = list(map(str, input().split()))

possibleCommittees = []
for i in range(peopleCount):
    user_input = list(map(str, input().split()))
    possibleCommittees.append(user_input)

maxPeoplePerCommitte = []
for j in range(committeesCount):
    user_input = int(input())
    maxPeoplePerCommitte.append(user_input)

"""
            peoples
            [P1, P2, P3, P4]

            committes
            [C1, C2, C3]

            possibleCommittees
            [['2', 'C1', 'C2'],
            ['2', 'C2', 'C3'],
            ['2', 'C1', 'C3'],
            ['3', 'C1', 'C2', 'C3']]

            maxPeoplePerCommittee
            [2, 3, 1]
            """

size_flow_matrix = peopleCount + committeesCount + 2

# outer for loop for rows, inner for loop for columns
flowMatrix = [[0 for _ in range(size_flow_matrix)] for _ in range(size_flow_matrix)]

# initial flow matrix with all zeros
# print('initial flow matrix with all zeros')
# print(flowMatrix)

# loop through rows of flow matrix
for i in range(size_flow_matrix):
    if i == 0:
        for k in range(peopleCount):
            # fill m_i between s and [P1, P2, P3, P4]
            flowMatrix[i][k + 1] = int(possibleCommittees[k][0])
    elif i > 0 and i <= peopleCount:
        # len(possibleCommittees[i - 1]) - 1 = max number of committees willing to be on by P_i
        # possibleCommittees[0] =
        for z in range(len(possibleCommittees[i - 1]) - 1):
            for k in range(committeesCount):
                if possibleCommittees[i - 1][z + 1] == committees[k]:
                    # fill 1 between P_i (i:1 to 4) and C_i (i:1 to 3) - filling people with the committes they are willing to go - fiiling with 1  because one committe should have different people
                    flowMatrix[i][peopleCount + 1 + k] = 1
    elif i > peopleCount and i <= peopleCount + committeesCount:
        # Z_i between C_i and t
        flowMatrix[i][size_flow_matrix - 1] = maxPeoplePerCommitte[i - peopleCount - 1]

# copy of flow matrix
temp_matrix = [[0 for _ in range(size_flow_matrix)] for _ in range(size_flow_matrix)]
for i in range(size_flow_matrix):
    for j in range(size_flow_matrix):
        temp_matrix[i][j] = flowMatrix[i][j]

# cefore maxflow
# print('before maxflow')
# print(flowMatrix)

max_flow = FordFulkerson(flowMatrix, size_flow_matrix - 1)

# after maxflow
# print('after maxflow')
# print(flowMatrix)

if max_flow == sum(maxPeoplePerCommitte):
    # This line creates a deep copy of the flowMatrix and assigns it to a new variable graph_np
    graph_np = [row[:] for row in flowMatrix]

    flow_matrix = [[0 for _ in range(size_flow_matrix)] for _ in range(size_flow_matrix)]

    for i in range(size_flow_matrix):
        for j in range(size_flow_matrix):
            # to compute the actual flow (assignments) in the flow network
            flow_matrix[i][j] = graph_np[i][j] - temp_matrix[i][j]

    # print('graph_np')
    # print(graph_np)
    # final flow matrix
    # print('final flow_matrix')
    # print(flow_matrix)


    committeePeople = []
    committeePeopleTmp = []
    for i in range(committeesCount):
        t = 0
        indexes = []
        indexes_temp = []
        for j in range(peopleCount):
            """
            In the context of flow networks and the Ford-Fulkerson algorithm, a flow value of -1 is often used to indicate that an edge is fully saturated, meaning it is at its maximum capacity.
            When an edge (corresponding to a possible assignment) is fully saturated, it signifies that a person has been assigned to a committee, and this assignment is "blocked" or "closed"
            because no more flow can pass through that edge.
            """
            # below if checks whether P_i has an edge with C_i that is -1
            if flow_matrix[j + 1][peopleCount + i + 1] == -1:
                indexes.append('P' + str(j + 1))
                indexes_temp.append(j + 1)
                t = t + 1
        committeePeople.append(indexes)
        committeePeopleTmp.append(indexes_temp)

    peopleCommittee = []
    for i in range(peopleCount):
        temp_list = []
        for j in range(committeesCount):
            for k in committeePeopleTmp[j]:
                if i + 1 == k:
                    temp_list.append('C' + str(j + 1))
        peopleCommittee.append(temp_list)

    print('Person: Committee')
    for i in range(peopleCount):
        print('P' + str(i + 1) + ":", end="")
        for j in peopleCommittee[i]:
            print(" " + j, end="")
        print("")

    print('\nCommittee: Person')
    for i in range(committeesCount):
        print('C' + str(i + 1) + ":", end="")
        for j in committeePeople[i]:
            print(" " + j, end="")
        print("")

else:
    print('Not Possible')

4
3
P1 P2 P3 P4
C1 C2 C3
2 C1 C2
2 C2 C3 
2 C1 C3
3 C1 C2 C3
2
3
1
initial flow matrix with all zeros
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]
before maxflow
[[0, 2, 2, 2, 3, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 0, 0, 0, 0, 3], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0]]
after maxflow
[[0, 0, 0, 1, 2, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 3, 1, 0]]
graph_np
[[0, 0, 0, 1, 2, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0,

In [ ]:
flowMatrix

[[0, 0, 0, 1, 2, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 2, 3, 1, 0]]

In [ ]:
possibleCommittees

[['2', 'C1', 'C2'],
 ['2', 'C2', 'C3'],
 ['2', 'C1', 'C3'],
 ['3', 'C1', 'C2', 'C3']]

In [ ]:
maxPeoplePerCommitte

[2, 3, 1]

In [ ]:
possibleCommittees[0]

['2', 'C1', 'C2']